Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
    http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# MONAI Label MONAIBUNDLE App - Multi-Organ Segmentation

***The Robust Workflow with Customized Models in MONAI Label***

In this notebook, we provide a monaibundle app usecase example with multi-organ segmentation. Using 13 abdominal organs segmentation with Swin UNETR network (bundle) as the demonstration, 3D Slicer as the client viewer, we show how MONAI Label workflow serves as interactive AI-Assisted tool for labeling multiple abdominal organs with bundle config files. 


**Multi-Organ Segmentation with Swin UNETR bundle:**

This tutorial uses a Swin UNETR bundle for the task of multi-organ segmentation task. The architecture of Swin UNETR is demonstrated as below
![workflow](./figures/monailabel_monaibundle_3dslicer/swinunetr.png)

**Target:** 13 abdominal organs including
- Spleen
- Right Kidney
- Left Kidney
- Gallbladder
- Esophagus
- Liver
- Stomach
- Aorta
- IVC
- Portal and Splenic Veins
- Pancreas
- Right adrenal gland
- Left adrenal gland

**Modality:** CT

The multi-organ segmentation with MONAI Label shown in 3D Slicer

![workflow](./figures/monailabel_monaibundle_3dslicer/multi_organ_seg_slicer.png)




## 1. Prepare MONAI Label

### Prerequisites
- **Install MONAI Label** weekly preview release: 


In [ ]:
!pip install monailabel-weekly

### 1.1 Download monaibundle app
Get and copy the monaibundle app using monailabel API 

In [ ]:
!monailabel apps --download --name monaibundle --output apps

### 1.2 Download sample data
The multi-organ segmentation task takes CT images as input. The bundle model Swin UNETR pre-trained weights are trained with Beyond The Cranial Vault (BTCV). In this tutorial, some sample CT images from MSD are used (for the purpose of app demonstration on Slicer 3D). Download MSD Task09 dataset as the sample dataset using monailabel API. 

**Note**: In this tutorial, we use some abdomen CT data for MONAI Label demostrations. If users want to use the BTCV dataset with ground truth annotations, please refer to [SwinUNETR 3D Segmentation](https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/swin_unetr_btcv_segmentation_3d.ipynb) for the dataset details.

In [ ]:
!monailabel datasets --download --name Task09_Spleen --output datasets

### 1.3 Starting MONAI Label Server

Specify the bundle name in --conf models <BUNDLENAME> argument. For example: swin_unetr_btcv_segmentation
    
```bash
--conf auth_token <Github Auth Token>
```

In [ ]:
!export BUNDLE_DOWNLOAD_SRC=ngc
!monailabel start_server --app apps/monaibundle --studies datasets/Task09_Spleen/imagesTr --conf models swin_unetr_btcv_segmentation --conf auth_token <YOUR AUTH TOKEN>

### 1.4 (Optional) File Structure for monaibundle app

To double check and confirm the correct monaibundle app and bundle files structure.

The directory tree structure should be:

```
└── monaibundle
    ├── __init__.py
    ├── lib
    │   ├── activelearning
    │   │   └── __init__.py
    │   ├── infers
    │   │   └── __init__.py
    │   ├── __init__.py
    │   └── trainers
    │       └── __init__.py
    ├── main.py
    ├── model
    │   └── swin_unetr_btcv_segmentation_v0.3.9
    │       ├── configs
    │       │   ├── evaluate.json
    │       │   ├── inference.json
    │       │   ├── logging.conf
    │       │   ├── metadata.json
    │       │   ├── multi_gpu_train.json
    │       │   └── train.json
    │       ├── docs
    │       │   ├── data_license.txt
    │       │   └── README.md
    │       ├── eval
    │       ├── LICENSE
    │       └── models
    │           └── model.pt
    ├── README.md
    └── requirements.txt
```


## 2. Auto Segmentation with 3D Slicer

Below is a step-by-step tutorial on using 3D Slicer for auto multi-organ segmentation with Swin UNETR bundle. Pre-trained model is provided by the bundle and loaded by MONAI Label server.


**Start 3D Slicer and MONAI Label Plugin**: the tutorial starts with monaibundle app in the 3D Slicer, please refer to the ["Hello, World!" tutorial](monailabel_HelloWorld_radiology_3dslicer.ipynb) for detailed instruction on installing 3D Slicer and MONAI Label plugin in Section 2.



### 2.1 Select Model and Fetch Data

- Select the bundle model loaded with MONAI Label server, "**swin_unetr_btcv_segmentation**" in this tutorial. **Note:** the model name can be different based on the downloaded bundle and can be changed and specified in the file system.

click **next sample** to load image data from datastore.

![slicer1](./figures/monailabel_monaibundle_3dslicer/swin_slicer1.png)


### 2.2 Run Auto Segmentation

- MONAI Label has loaded the pre-trained weights within the bundle for multi-organ segmentation task, click **run** to do inference on the currently loaded data. 

- Note: the pre-trained models are in the bundle's "**models**" folder. For instance, in this usecase, a downloaded pre-trained model is saved at "**apps/monaibundle/model/{BUNDLENAME}/models/model.pt**".

- Note: Swin UNETR is a large model, by default it will take ~10G memory for inference, modify the bundle config file if out of memory.

Users can monitor the logs in the MONAI Label server terminal

![slicer1](./figures/monailabel_monaibundle_3dslicer/swin_slicer2.png)


### 2.3 Edit Annotations and Submit Label

- Users can edit the inference labels manually using Segment Editor, the tools for adjusting annotation. Once done annotation, click **Submit Label** to save the ground truth label to the file system.

- Note: the final annotations will be saved to "**labels/final**" folder in the study dataset, for example, in this usecase, the ground truth label will be saved to "**datasets/Task09_Spleen/imagesTr/labels/final**"

#### Corrections with Segment Editor:

![slicer1](./figures/monailabel_monaibundle_3dslicer/swin_slicer3.png)


## 3. Active Learning with 3D Slicer

Active learning and interactive fine-tuning models have highlighted features in MONAI Label, users can train their models anytime when new annotated labels are saved. Click **Train** button, MONAI Label server will fetch saved final ground truth label and fine-tune the prior model. 

Users can monitor the logs in the MONAI Label server terminal

![slicer1](./figures/monailabel_monaibundle_3dslicer/swin_slicer4.png)

Same to the Radiology app, users can do auto segmentation using the latest fine-tuned model. 
The active learning process will select unlabeled batch data.

The next image is selected, already trained images will be marked as labeled data, then won't be selected for the next labeling batch. Active learning strategies such as "first/random" will be used for selecting which unlabeled data to fetch. 



## 4. Repeat Until All Data are Annotated and Trained 

Repeated fetch data and active learning process to section 2.2, until all unlabeled data are annotated and trained. 

For each training loop, the new best metric model will be saved in "**model/model.pt**", in this usecase, ""**apps/monaibundle/{BUNDLENAME}/model/model.pt**" is saved.

## 5. Conclusion

This tutorial demonstrates MONAI Label usage with monaibundle app, the multi-organ segmentation with Swin UNETR bundle is introduced. Active learning process, auto segmentation, segmentation editor, submit labels, and save models are included with the bundle. The monaibundle app highlights the robust model deployment for MONAI Label.